In [9]:
import os
import json
import pandas as pd
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

json_dir = "ipl_json"
deliveries_data = []

In [10]:
def extract_deliveries_info(json_file):
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)
            match_info = f"{data['info']['dates'][0]}_{data['info']['teams'][0]}_{data['info']['teams'][1]}"
            innings = data.get('innings', [])
            for inning_number, inning in enumerate(innings, start=1):
                batting_team = inning['team']
                bowling_team = data['info']['teams'][0] if data['info']['teams'][0] != batting_team else data['info']['teams'][1]
                for over_number, over in enumerate(inning.get('overs', []), start=1):
                    for ball, delivery in enumerate(over.get('deliveries', []), start=1):
                        batsman = delivery['batter']
                        non_striker = delivery['non_striker']
                        bowler = delivery['bowler']
                        total_runs = delivery['runs']['total']
                        player_dismissed = None
                        if 'wickets' in delivery:
                            player_dismissed = delivery['wickets'][0]['player_out']
                        
                        delivery_info = {
                            'Match_ID': match_info,
                            'Inning': inning_number,
                            'Batting_Team': batting_team,
                            'Bowling_Team': bowling_team,
                            'Over': over_number,
                            'Ball': ball,
                            'Batsman': batsman,
                            'Non_Striker': non_striker,
                            'Bowler': bowler,
                            'Total_Runs': total_runs,
                            'Player_Dismissed': player_dismissed
                        }
                        deliveries_data.append(delivery_info)
    except Exception as e:
        print(f"Error processing file '{json_file}': {str(e)}")

# Process each JSON file in the directory
for filename in os.listdir(json_dir):
    if filename.endswith(".json"):
        json_file_path = os.path.join(json_dir, filename)
        extract_deliveries_info(json_file_path)

deliveries_df = pd.DataFrame(deliveries_data)
deliveries_df[['Match_Date', 'Team1', 'Team2']] = deliveries_df['Match_ID'].str.split('_', expand=True)
deliveries_df['Match_Date'] = pd.to_datetime(deliveries_df['Match_Date'])
deliveries_df = deliveries_df.sort_values(by=['Match_Date', 'Match_ID', 'Inning', 'Over', 'Ball'], ascending=[False, True, True, True, True])
deliveries_df = deliveries_df.reset_index(drop=True)
deliveries_df = deliveries_df.drop(columns=['Team1', 'Team2'])
deliveries_df = deliveries_df[['Match_Date', 'Inning', 'Batting_Team', 'Bowling_Team', 'Over', 'Ball', 'Batsman', 'Non_Striker', 'Bowler', 'Total_Runs', 'Player_Dismissed', 'Match_ID']]
deliveries_df.to_csv('deliveries_info.csv', index=False)